In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img
import random
from os import listdir
from os.path import isfile, join
import keras.callbacks as kcall

In [ ]:
# GPU config
# import tensorflow as tf
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#   except RuntimeError as e:
#     print(e)

In [ ]:
train_dir = '../input/train/'
test_dir = '../input/test/'
labels = ['pneumonia', 'COVID-19', 'normal']
img_width, img_height, channels = 512, 512, 3
color_mode = 'rgb'
batch_size = 8
epochs = 50

In [ ]:
datagen = ImageDataGenerator(
                    samplewise_center=True,
                    samplewise_std_normalization=True
                    )

train_generator = datagen.flow_from_directory(train_dir, target_size = (img_width, img_height), batch_size = batch_size, color_mode = color_mode, class_mode='categorical')
test_generator = datagen.flow_from_directory(test_dir, target_size = (img_width, img_height), batch_size = batch_size, color_mode = color_mode, class_mode='categorical')

train_size = 13897
test_size = 1579

In [ ]:
from keras.applications import InceptionV3, ResNet50V2, DenseNet201
from keras.optimizers import Adam

inception = InceptionV3(
    include_top=True,
    weights=None,
    input_shape=(img_width, img_height, channels),
    classes=3
)

resNet = ResNet50V2(
    include_top=True,
    weights=None,
    input_shape=(img_width, img_height, channels),
    classes=3
)

denseNet = DenseNet201(
    include_top=True,
    weights=None,
    input_shape=(img_width, img_height, channels),
    classes=3
)

# incleption.summary()
# resNet.summary()
# denseNet.summary()
inception.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
resNet.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
denseNet.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
inception_history = inception.fit(train_generator,
        steps_per_epoch = train_size // batch_size,
        epochs = epochs,
        validation_data = test_generator)
        
resNet_history = resNet.fit(train_generator,
        steps_per_epoch = train_size // batch_size,
        epochs = epochs,
        validation_data = test_generator)

denseNet_history = denseNet.fit(train_generator,
        steps_per_epoch = train_size // batch_size,
        epochs = epochs,
        validation_data = test_generator)

In [ ]:
plt.grid(True)
plt.xlabel("Эпохи")
plt.ylabel("Доля правильных ответов")
plt.plot(inception_history['accuracy'])
plt.plot(resNet_history['accuracy'])
plt.plot(denseNet_history['accuracy'])
plt.legend(['Inception', 'ResNet', 'DenseNet'])
plt.savefig("train_accuracy.pdf")

plt.grid(True)
plt.xlabel("Эпохи")
plt.ylabel("Доля правильных ответов")
plt.plot(inception_history['val_accuracy'])
plt.plot(resNet_history['val_accuracy'])
plt.plot(denseNet_history['val_accuracy'])
plt.legend(['Inception', 'ResNet', 'DenseNet'])
plt.savefig("val_accuracy.pdf")

In [ ]:
# #Evaluate the model's perfomance
# performance = model.evaluate_generator(test_generator)
# print("Loss on Test Set: %.2f" % (performance[0]))
# print("Accuracy on Test Set: %.2f" % (performance[1]*100))